## Instalar dependencias:


In [1]:
# !python -m pip install Cython
# !python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
# !python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
# !python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
# !python -m pip install pyarrow
# !python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

## Importar módulos:


In [2]:
import numpy as np
import pandas as pd

In [3]:
#Constructor de curvas de luz
from lc_classifier.utils import LightcurveBuilder
#Compositor de Extractores
from lc_classifier.features import FeatureExtractorComposer
#Extractores Generales:
from lc_classifier.features import MHPSExtractor
from lc_classifier.features import PeriodExtractor
from lc_classifier.features import GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor
from lc_classifier.features import IQRExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor
#Extractores Supernova:
from lc_classifier.features import SupernovaeDetectionFeatureExtractor
from lc_classifier.features import SupernovaeDetectionAndNonDetectionFeatureExtractor
from lc_classifier.features import SNParametricModelExtractor
from lc_classifier.features import ZTFColorFeatureExtractor

from lc_classifier.features import GalacticCoordinatesExtractor
from lc_classifier.features import RealBogusExtractor



## Importar datasets

In [4]:
#Definir dataframes
df_novas = pd.read_pickle('df_novas.pkl')

In [5]:
df_novas.head()

,oid,fid,magpsf,sigmapsf,magpsf_corr,sigmapsf_corr,mjd,isdiffpos,rb,ra,dec,sgscore1,classALeRCE
0,ZTF18aahvndq,1,15.385100,0.028791,15.385100,0.028791,58991.302095,1.0,0.770000,198.722678,27.008621,0.175976,SNIa
1,ZTF18aahvndq,1,15.712121,0.031231,15.712121,0.031231,59000.260914,1.0,0.894286,198.722679,27.008635,0.175976,SNIa
2,ZTF18aahvndq,1,16.072975,0.032161,16.072975,0.032161,59004.192674,1.0,0.907143,198.722653,27.008647,0.175976,SNIa
3,ZTF18aahvndq,1,17.708502,0.049931,17.708502,0.049931,59021.227604,1.0,0.947143,198.722660,27.008624,0.175976,SNIa
4,ZTF18aahvndq,1,15.617474,0.028520,15.617474,0.028520,58986.223646,1.0,0.837143,198.722660,27.008636,0.175976,SNIa


## Definir extractor de características (**Feature Extractor**)

In [6]:
bands = ['g', 'r']

feature_extractor = FeatureExtractorComposer(
    [   
        SupernovaeDetectionFeatureExtractor(bands),
        MHPSExtractor(bands), #Esta
        PeriodExtractor(bands, 0.01, 0.99, False, None, 20), #ESTA
        GPDRWExtractor(bands), #ESTA
        FoldedKimExtractor(bands), #ESTA
        HarmonicsExtractor(bands), #ESTA
        IQRExtractor(bands), #Esta
        PowerRateExtractor(bands), #ESTA
        TurboFatsFeatureExtractor(bands), #Esta
        # SNParametricModelExtractor(bands), #  ESTA
        # ZTFColorFeatureExtractor(), #Esta
        # GalacticCoordinatesExtractor(), #Esta
        # RealBogusExtractor(), #Esta
        # SupernovaeDetectionAndNonDetectionFeatureExtractor(bands), #ESTA
    ]
)


## Definir Dataset Curvas de Luz

In [7]:
df_lightcurves = pd.DataFrame()
df_lightcurves['oid'] = df_novas["oid"]
df_lightcurves['band'] = df_novas['fid'].replace({1 : 'g', 2 : 'r'})
df_lightcurves['time'] = df_novas['mjd']
df_lightcurves['magnitude'] = df_novas['magpsf_corr']
df_lightcurves['error'] = df_novas['sigmapsf_corr']
df_lightcurves['isdiffpos'] = df_novas['isdiffpos']

df_lightcurves['magpsf'] = df_novas['magpsf']
df_lightcurves['sigmapsf'] = df_novas['sigmapsf']
df_lightcurves['rb'] = df_novas['rb']
df_lightcurves['ra'] = df_novas['ra']
df_lightcurves['dec'] = df_novas['dec']


# 'rb', 'error', 'magpsf', 'band', 'sigmapsf', 'time', 'ra', 'magnitude', 'dec', 'isdiffpos'

df_lightcurves = df_lightcurves.set_index('oid')

df_lightcurves.head()

,band,time,magnitude,error,isdiffpos,magpsf,sigmapsf,rb,ra,dec
oid,,,,,,,,,,
ZTF18aahvndq,g,58991.302095,15.385100,0.028791,1.0,15.385100,0.028791,0.770000,198.722678,27.008621
ZTF18aahvndq,g,59000.260914,15.712121,0.031231,1.0,15.712121,0.031231,0.894286,198.722679,27.008635
ZTF18aahvndq,g,59004.192674,16.072975,0.032161,1.0,16.072975,0.032161,0.907143,198.722653,27.008647
ZTF18aahvndq,g,59021.227604,17.708502,0.049931,1.0,17.708502,0.049931,0.947143,198.722660,27.008624
ZTF18aahvndq,g,58986.223646,15.617474,0.028520,1.0,15.617474,0.028520,0.837143,198.722660,27.008636


## Extraer features:

In [ ]:
features = feature_extractor.compute_features(df_lightcurves)
features

/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: overflow encountered in square
  return np.sum(alpha ** 2 / self._d)
/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(alpha ** 2 / self._d)
/home/jovyan/work/mds/Proyecto/src/lc-classifier/lc_classifier/features/extractors/gp_drw_extractor.py:52: RuntimeWarning: overflow encountered in exp
  theta = np.exp(params)
/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(alpha ** 2 / self._d)
/opt/conda/lib/python3.9/site-packages/celerite2/core.py:427: RuntimeWarning: invalid value encountered in double_scalars
  return self._norm - 0.5 * self._do_norm(y - self._mean_value)
/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: overflow encountered in square
  return np.sum(alpha ** 2 / self._d)
/home/jovyan/work/mds/Pr

In [ ]:
features = features.reset_index()
df_limpio = pd.merge(features, df_novas[['oid','classALeRCE']], left_on='oid', right_on='oid')

In [ ]:
df_limpio

## Exportar Features

In [ ]:
df_limpio.to_csv('features_extraidas.csv', sep=',')